In [2]:
import rasterio

In [1]:
image_path = 'data/batch_001/8928dec4ddbffff/DJI_0876.JPG'

In [3]:
# import the image
with rasterio.open(image_path) as src:
    # read the entire image
    img = src.read()

/Users/kevinlacaille/.pyenv/versions/3.11.4/envs/scipy2024-env/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [18]:
import exiftool
import json

In [19]:
# Extract the metadata from the image
with exiftool.ExifTool() as et:
    metadata = json.loads(et.execute(b'-j', image_path))


In [7]:
print(metadata)

[{
  "SourceFile": "data/batch_001/8928dec4ddbffff/DJI_0876.JPG",
  "ExifTool:ExifToolVersion": 12.60,
  "ExifTool:Warning": "[minor] Possibly incorrect maker notes offsets (fix by 1783?)",
  "File:FileName": "DJI_0876.JPG",
  "File:Directory": "data/batch_001/8928dec4ddbffff",
  "File:FileSize": 5257518,
  "File:FileModifyDate": "2024:06:14 03:25:40-07:00",
  "File:FileAccessDate": "2024:07:02 10:35:17-07:00",
  "File:FileInodeChangeDate": "2024:06:13 20:28:40-07:00",
  "File:FilePermissions": 100664,
  "File:FileType": "JPEG",
  "File:FileTypeExtension": "JPG",
  "File:MIMEType": "image/jpeg",
  "File:ExifByteOrder": "II",
  "File:ImageWidth": 4000,
  "File:ImageHeight": 3000,
  "File:EncodingProcess": 0,
  "File:BitsPerSample": 8,
  "File:ColorComponents": 3,
  "File:YCbCrSubSampling": "2 1",
  "EXIF:ImageDescription": "DCIM\\108MEDIA\\DJI_0876.JPG",
  "EXIF:Make": "DJI",
  "EXIF:Model": "FC3682",
  "EXIF:Orientation": 1,
  "EXIF:XResolution": 72,
  "EXIF:YResolution": 72,
  "EXIF:R

In [36]:
# Extract the GPS Altitude
altitude = metadata[0].get("EXIF:GPSAltitude")
focal_length = metadata[0].get("EXIF:FocalLength")  # in mm
# Size of pixel = sensor width (m) / image width (px)
pixel_pitch = 6.17e-3 / src.width  # sensor width known to be 1/2.3” = 6.17mm

print("Altitude: ", altitude, "meters")
print("Focal Length: ", focal_length, "mm")
print("Pixel Pitch: ", pixel_pitch, "meters / px")

Altitude:  125.09 meters
Focal Length:  6.72 mm
Pixel Pitch:  1.5425e-06 meters / px


In [37]:
# Calculate the GSD (Ground Sample Distance)
gsd = (altitude * pixel_pitch) / (focal_length / 1000)
print("GSD: ", gsd, "meters / px")
print("GSD: ", gsd * 100, "cm / px")


GSD:  0.028712994791666672 meters / px
GSD:  2.8712994791666673 cm / px


In [40]:
import numpy as np

height_tree = 10  # meters

# GSD at the tree height
gsd = (altitude - height_tree) * pixel_pitch / focal_length

diameter_of_tree = 10  # meters
diameter_of_tree_pixels = diameter_of_tree / gsd
area_of_tree = np.pi * (diameter_of_tree_pixels / 2)**2
print(f'Area of tree: {area_of_tree:.2f} pixels')

Area of tree: 112538961171.59 pixels
